<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/predictions/Prediction_Offense_Final1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install pmdarima
!pip install category_encoders==2.0.0

In [2]:
# Imports
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from sklearn import preprocessing

import category_encoders as ce

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Import data
original_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_offense.csv')
kickers_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_kicker.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_offense.csv')
player_df = pd.concat([kickers_df, offense_df], ignore_index=True)

In [4]:
# The dataframe of actual offensive points for each game from 2000-2019
original_df.head()

,player,first,last,name,position1,2000-game1,2000-game2,2000-game3,2000-game4,2000-game5,2000-game6,2000-game7,2000-game8,2000-game9,2000-game10,2000-game11,2000-game12,2000-game13,2000-game14,2000-game15,2000-game16,2001-game1,2001-game2,2001-game3,2001-game4,2001-game5,2001-game6,2001-game7,2001-game8,2001-game9,2001-game10,2001-game11,2001-game12,2001-game13,2001-game14,2001-game15,2001-game16,2002-game1,2002-game2,2002-game3,...,2017-game9,2017-game10,2017-game11,2017-game12,2017-game13,2017-game14,2017-game15,2017-game16,2018-game1,2018-game2,2018-game3,2018-game4,2018-game5,2018-game6,2018-game7,2018-game8,2018-game9,2018-game10,2018-game11,2018-game12,2018-game13,2018-game14,2018-game15,2018-game16,2019-game1,2019-game2,2019-game3,2019-game4,2019-game5,2019-game6,2019-game7,2019-game8,2019-game9,2019-game10,2019-game11,2019-game12,2019-game13,2019-game14,2019-game15,2019-game16
0,AD-0100,Andy,Dalton,Andy Dalton,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.6,18.56,19.16,19.26,7.64,0.52,11.42,21.68,16.52,26.60,14.08,24.78,12.92,17.16,7.92,20.20,6.92,19.34,8.10,0.00,0.00,0.00,0.00,0.00,21.72,19.64,16.06,3.64,19.98,13.60,18.34,17.16,0.00,0.00,0.00,13.32,9.58,2.14,33.84,16.60
1,AM-1150,A.J.,McCarron,A.J. McCarron,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,1.88,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.32,0.00,0.00,-0.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.90
2,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,21.90,0.00,0.00,25.94,16.04,20.90,15.02,28.68,28.40,16.94,19.16,18.06,22.58,11.92,15.32,20.24,13.46,43.88,1.04,12.92,14.36,14.30,25.48,9.42,18.32,44.76,28.10,12.94,10.02,9.46,28.12,11.40,14.42,9.34,19.02
3,AT-0160,Alex,Tanney,Alex Tanney,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00
4,BA-0375,Brandon,Allen,Brandon Allen,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.02,13.40,2.08,0.00,0.00,0.00,0.00,0.00


In [5]:
# The dataframe of all 2019 offensive players (kickers and offense)
player_df.head(50)

,player,first,last,name,position1,start
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,1996
1,MB-4600,Matt,Bryant,Matt Bryant,K,2002
2,RG-1500,Robbie,Gould,Robbie Gould,K,2005
3,MN-0800,Mike,Nugent,Mike Nugent,K,2005
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,2006
5,MP-2100,Matt,Prater,Matt Prater,K,2006
6,MC-3000,Mason,Crosby,Mason Crosby,K,2007
7,NF-0300,Nick,Folk,Nick Folk,K,2007
8,SH-0400,Stephen,Hauschka,Stephen Hauschka,K,2008
9,RS-3400,Ryan,Succop,Ryan Succop,K,2009


In [0]:
# Add a row to the final_df dataframe
# Each row represents the predicted points for each team
def add_row(df, p, f, l, n, pos, pred, act):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position': pos,
                    'week1-pred': pred,
                    'week1-act': act
                    }, ignore_index=True)
    
    return df

In [7]:
# The main code for iterating through the player(offense and kicker) list, calculating the points and adding the rows
# to the final_df dataframe.
column_names = ['player',
                'first',
                'last',
                'name',
                'position', 
                'week1-pred',
                'week1-act'
                 ]

player_list = offense_df['player'].tolist()

final_df = pd.DataFrame(columns = column_names)

for player in player_list:

    first = player_df['first'].loc[(player_df['player']==player)].iloc[0]
    last = player_df['last'].loc[(player_df['player']==player)].iloc[0]
    name = player_df['name'].loc[(player_df['player']==player)].iloc[0]
    position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]
    start_year = player_df['start'].loc[(player_df['player']==player)].iloc[0]
    row = original_df.index[(original_df['player']==player)][0]

    if start_year < 2000:
        start_year = 2000
    col = ((start_year - 2000) * 16) + 5
    train_data = original_df.iloc[row, col:309]
    actuals = original_df.iloc[row, 309:325]
    act_points = actuals.sum()

    print(player)

    if (start_year < 2016) & (player != 'GG-0310'):
        # ARIMA model
        model = pm.auto_arima(train_data, start_p=1, start_q=1,
                              test='adf',       # use adftest to find optimal 'd'
                              max_p=3, max_q=3, # maximum p and q
                              m=1,              # frequency of series
                              d=None,           # let model determine 'd'
                              seasonal=False,   # No Seasonality
                              start_P=0, 
                              D=0, 
                              trace=False,
                              error_action='ignore',  
                              suppress_warnings=True, 
                              stepwise=True)
    
        # Forecast
        n_periods = 16
        fc = model.predict(n_periods=n_periods, return_conf_int=False)
        index_of_fc = np.arange(len(train_data), len(train_data)+n_periods)
        fc_series = pd.Series(fc, index=index_of_fc)
        pred_points = fc_series.sum()
    else:
        pred_points = train_data.mean() * 16

    final_df = add_row(final_df, player, first, last, name, position1, pred_points, act_points)

TB-2300
DB-3800
JM-2900
BR-1100
EM-0200
MS-0200
PR-0300
AR-1300
RF-0500
MM-4700
JF-1900
MR-2500
BH-1900
CD-0300
MS-4100
CM-1500
AD-0100
CN-0500
TT-0500
AT-0160
CK-0250
KC-2350
NF-0250
RG-1850
RT-0150
RW-3850
MB-0450
MG-0850
RG-1885
AM-1150


/usr/local/lib/python3.6/dist-packages/pmdarima/arima/auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


BB-2425
DC-0725
GG-0310
JG-0650
TB-2650
BH-2300
JW-5650
MM-0912
SM-0425
TS-1450
BA-0375
CW-1425
DP-2037
JB-5725
JD-2850
JG-1850
DW-1725
MT-1800
NM-0687
PM-0025
TH-1625
BM-0650
JA-1087
JR-3250
KA-0737
LF-0025
LJ-0150
MR-2350
SD-0175
TB-2093
DS-3300
FG-0200
AP-0700
ML-2500
LM-1000
JD-1650
AS-0900
BP-1200
DL-1700
MI-0100
NB-0200
PD-0700
TJ-2400
AM-2850
BB-2250
JO-0550
CA-0750
CT-1220
GB-0350
KB-0350
KJ-1400
LB-0250
LM-1850
RB-4670
SW-0350
ZL-0200
CH-5000
DF-1875
DW-3125
JW-2550
KS-1450
RN-1250
SP-0850
AA-0025
DJ-1325
DJ-1850
JA-0450
JA-1025
MB-3862
MB-5175
MD-0150
MG-1150
RM-5050
TC-2050
TG-1950
TM-3150
TY-0150
ZZ-0100
AJ-0375
CH-0525
CP-2550
DB-2950
DF-1375
DH-2450
DV-0550
DW-1350
DW-1550
DW-1750
EE-0250
EP-0350
JF-1250
JH-5575
JR-2075
JW-4625
KD-1462
PB-0125
PP-0350
TC-0450
TE-0350
TP-1950
WS-0650
AA-0557
AE-0550
AJ-1143
AK-0050
BH-1225
CC-0750
CC-2350
CM-1225
DC-3550
DO-0025
JC-4075
JM-5275
JW-3650
KH-2850
LF-0650
MB-3537
MM-0050
PR-0125
RO-0550
SP-0825
TC-1975
TE-0025
TL-1037
WG-0050


/usr/local/lib/python3.6/dist-packages/pmdarima/arima/auto.py:328: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


DT-0850
JB-6950
JL-0215
JM-1550
ME-0600
ML-0650
OB-0075
PR-0150
QE-0100
RG-1650
SR-0750
SW-0450
TG-0025
TJ-2350
WS-0925
AC-2350
AH-3862
BP-0450
CC-3950
DB-6375
DC-1050
DF-2200
DJ-0850
DP-0350
DS-3031
DW-2350
GD-0050
JC-5680
JH-1350
JK-2250
JN-0625
NA-0050
PD-0800
SD-0537
TL-1025
TW-2875
AE-1037
CC-4450
CM-2675
CR-1525
DR-2225
DW-2850
GA-0250
JG-2950
JH-5125
JM-3475
JM-4175
KR-0312
LT-0862
MH-0225
MJ-1450
MT-0875
MT-0887
PC-0625
RA-1125
RH-1950
SS-1250
TB-2075
TD-0737
TH-1750
TJ-2650
TS-1262
WF-0300
AC-0387
AP-0388
BW-2475
CB-1750
CD-0975
CG-1425
CG-2162
CH-3775
CK-1300
CS-0050
CW-2450
DM-3050
DW-2275
FB-0750
GW-0250
IF-0175
IM-0125
JN-0350
JR-1950
JR-3280
JS-4750
KB-1625
KC-1550
KG-0650
KH-2075
MH-3050
MW-2825
RD-0150
RD-0450
RS-3525
TP-0675
TT-0150
ZJ-0100
ZP-0050
AC-0237
AL-0387
AM-1975
AT-0247
BB-1950
BP-1375
BZ-0100
CB-1152
CB-1650
CK-0825
CL-0125
CM-1650
CR-0862
CS-1925
CS-4462
CW-3250
DB-5912
DC-0025
DC-1418
DH-1050
DM-2850
DP-1263
DR-0550
DS-0875
JS-1350
JW-0934
JW-1050
JW-5475


In [0]:
# The final_df dataframe
final_df['week1-diff'] = final_df['week1-pred'] - final_df['week1-act']
final_df['week1-pct'] = final_df['week1-diff']/final_df['week1-pred']
final_df['week1-pred'] = final_df['week1-pred'].astype(int)

In [9]:
# Calculate the metrics
pred_median_error = final_df['week1-pct'].median()

print('Median Error - %.4f%%' % (pred_median_error * 100))

Median Error - -2.5282%


In [10]:
final_df.head(50)

,player,first,last,name,position,week1-pred,week1-act,week1-diff,week1-pct
0,TB-2300,Tom,Brady,Tom Brady,QB,261,265.68,-3.972314,-1.517844e-02
1,DB-3800,Drew,Brees,Drew Brees,QB,258,227.76,30.733579,1.188949e-01
2,JM-2900,Josh,McCown,Josh McCown,QB,58,0.76,57.502193,9.869555e-01
3,BR-1100,Ben,Roethlisberger,Ben Roethlisberger,QB,237,12.74,224.894588,9.463883e-01
4,EM-0200,Eli,Manning,Eli Manning,QB,226,56.38,169.842112,7.507759e-01
5,MS-0200,Matt,Schaub,Matt Schaub,QB,0,32.90,-32.965319,5.046821e+02
6,PR-0300,Philip,Rivers,Philip Rivers,QB,275,240.50,35.303691,1.280030e-01
7,AR-1300,Aaron,Rodgers,Aaron Rodgers,QB,295,282.38,13.352208,4.514966e-02
8,RF-0500,Ryan,Fitzpatrick,Ryan Fitzpatrick,QB,120,242.46,-122.134927,-1.015041e+00
9,MM-4700,Matt,Moore,Matt Moore,QB,27,45.26,-17.628646,-6.379943e-01


In [0]:
# Save the results to .csv file
final_df.to_csv('/content/week1-pred-offense-norookies.csv', index=False)